# Backup Management
This notebook will demonstrate the platform APIs for backup management. 

## Setup

Set the `YB_PLATFORM_URL` and `YB_API_KEY` environment variables in a .env file before running this notebook.

Note: "verify=False" is used to disable SSL verification for the purposes of this demonstration, but you should use appropriate certificates in a production environment.


In [ ]:
import os
import requests
from pprint import pprint
from dotenv import load_dotenv

# just for demo environment, not production
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

load_dotenv()

yba_url = os.getenv("YBA_PLATFORM_URL")
yba_api_key = os.getenv("YBA_API_KEY")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{yba_api_key}"
}

session = requests.Session()
session.verify = False
session.headers = headers

universe_uuid="e4d77b12-3bec-4965-a625-056a1fc0ef85" # Replace with your universe UUID

## Get the customer ID

The customer ID is unique to the YBA platform and is required for the following Platform API calls. 

In [ ]:
route = f"{yba_url}/api/v1/session_info"
response = session.get(url=route).json()
customer_uuid = response["customerUUID"]

## Create an ad hoc backup

Create a new backup for a specific universe.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/2d87d8d6901ad-create-a-backup-v2

In [ ]:
keyspace = "dss" # Replace with your keyspace
storage_config_uuid = "e78ab1c8-99af-4bfb-9423-4978955b5dcb" # Replace with your storage config UUID
time_before_delete = "86400000" # 1 day, in milliseconds
expiry_time_unit = "MILLISECONDS"
backup_type="PGSQL_TABLE_TYPE"

route = f"{yba_url}/api/v1/customers/{customer_uuid}/backups"

payload = {
  "universeUUID": universe_uuid,
  "keyspace": keyspace,
  "storageConfigUUID": storage_config_uuid,
  "timeBeforeDelete": time_before_delete,
  "expiryTimeUnit": expiry_time_unit,
  "backupType": backup_type
}

# avoids no CSRF token error by emptying the cookie jar
session.cookies = requests.cookies.RequestsCookieJar()

response = session.post(url=route, json=payload).json()

if response.get('success', True):
  task_uuid = response['taskUUID']
  print(f"Task UUID: {response['taskUUID']}")
else:
  print(f"Request failed:\n{response}")

## Check on this task running in the background

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/ae83717943b4c-get-a-task-s-status

In [ ]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/tasks/{task_uuid}"
response = session.get(url=route).json()
pprint(response)

## Get the backup ID associated with this task

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/bcec8c9a18e02-list-backups-associated-with-a-task

In [ ]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/universes/{universe_uuid}/backups/tasks/{task_uuid}"
response = session.get(url=route).json()

for item in response:
    if item['taskUUID'] == task_uuid:
        backup_uuid = item['backupUUID']
        print(f"Backup UUID: {backup_uuid}")
        break
else:
    print(f"No backup found for task UUID {task_uuid}")

## List existing backups

List existing backups for the selected universe.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/a1b711e0f6405-list-backups-paginated-v2

In [ ]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/universes/{universe_uuid}/backups"
response = session.get(url=route).json()

for item in response:
    print(f"Backup UUID: {item['backupUUID']}, expires: {item['expiry']}")

## Show expiry time for one backup

Show expiry time for a backup.

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/9a35f53fa6a1c-get-backup-v2

In [ ]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/backups/{backup_uuid}"
response = session.get(url=route).json()
pprint(response["expiry"])

## Delete a backup

Delete a backup (before it expires).

ref: https://api-docs.yugabyte.com/docs/yugabyte-platform/e7527b334c71a-delete-backups-v2

In [ ]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/backups/delete"

payload = { "deleteBackupInfos": [ {"backupUUID": backup_uuid} ] }

# avoids no CSRF token error by emptying the cookie jar
session.cookies = requests.cookies.RequestsCookieJar()

response = session.post(url=route, json=payload).json()
task_uuid = response.get('taskUUID', [None])[0]
print(f"Task UUID: {task_uuid}")

## Check on the backup deletion task

In [ ]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/tasks/{task_uuid}"
response = session.get(url=route).json()
pprint(response)